In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Activation
import numpy as np
import time
import pandas as pd

data_seq_dim = 50 #our sequence data is dimension 50 (maybe this is number of new accounts opened for each account type etc)
data_x_dim = 25 #our non-sequence data 
timesteps = 12 #last 12 months of sequence data (12 time steps)
nb_classes = 3 #classes 
batch_size = 32 

# generate training data (10 mini-batches for a single epoch)
x_seq_train = np.random.random((batch_size * 10, timesteps, data_seq_dim))
x_train = np.random.random((batch_size * 10, timesteps,data_x_dim))

#one-hot classes
y_train=np.zeros((batch_size*10, timesteps,nb_classes))
for i in range(batch_size*10):
    for j in range(timesteps):
        y_train[i,j,np.random.randint(nb_classes)]=1
  


Using Theano backend.


In [57]:
print 'X sequence dim: ', x_seq_train.shape 
print 'X non sequence dim: ', x_train.shape
print 'y dim: ', y_train.shape


X sequence dim:  (320L, 12L, 50L)
X non sequence dim:  (320L, 12L, 25L)
y dim:  (320L, 12L, 3L)


In [59]:
#vanilla LSTM on the sequence data

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(timesteps, data_seq_dim)))
model.add(TimeDistributed(Dense(nb_classes))) #nb_classes outputs for this dense layer
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop')
model.fit(x_seq_train, y_train, batch_size=batch_size, nb_epoch=100, show_accuracy=True)
time.sleep(0.1) #https://github.com/fchollet/keras/issues/2110


C:\Users\machine\Anaconda2\lib\site-packages\keras-1.0.2-py2.7.egg\keras\models.py:392: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


Epoch 1/100
320/320 [==============================] - 0s - loss: 1.1130     
Epoch 2/100
320/320 [==============================] - 0s - loss: 1.1002     
Epoch 3/100
320/320 [==============================] - 0s - loss: 1.1005     
Epoch 4/100
320/320 [==============================] - 0s - loss: 1.0990     
Epoch 5/100
320/320 [==============================] - 0s - loss: 1.0980     
Epoch 6/100
320/320 [==============================] - 0s - loss: 1.0958     
Epoch 7/100
320/320 [==============================] - 0s - loss: 1.0929     
Epoch 8/100
320/320 [==============================] - 0s - loss: 1.0975     
Epoch 9/100
320/320 [==============================] - 0s - loss: 1.0933     
Epoch 10/100
320/320 [==============================] - 0s - loss: 1.0898     
Epoch 11/100
320/320 [==============================] - 0s - loss: 1.0901     
Epoch 12/100
320/320 [==============================] - 0s - loss: 1.0898     
Epoch 13/100
320/320 [==============================] - 0s - 

NameError: name 'time' is not defined

In [20]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display

In [21]:
#Inspired by: https://plot.ly/python/big-data-analytics-with-pandas-and-sqlite/


full=pd.read_csv('https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail.csv',encoding='latin-1')
full['invoice_date'] = pd.to_datetime(full['InvoiceDate']) # Convert to datetimes
full['invoice_date']=full['invoice_date'].dt.date #just date
full.drop(['InvoiceDate'],axis=1)

disk_engine = create_engine('sqlite:///C:\\Users\\machine\\Documents\\data\\dataonline_retail.db')
full.to_sql('data', disk_engine, if_exists='replace')

In [ ]:
full.head()

In [23]:
aug11_custs = pd.read_sql_query(
    " select a.*, strftime('%m',invoice_date) as month, strftime('%Y',invoice_date) as year, strftime('%W',invoice_date) as week  "
    " from data a inner join "
    "(SELECT distinct customerID "
                       "FROM data "
                  "where invoice_date >= '2010-12-01' and invoice_date <= '2011-08-31' ) b "
    "on a.customerID = b.customerID "
                       , disk_engine)


Sep11_custs = pd.read_sql_query(
    " SELECT distinct customerID "
                       "FROM data "
                  "where invoice_date >= '2011-09-01' and invoice_date <= '2011-11-30' " 
                       , disk_engine)



#group by month 


In [127]:
pd.read_sql_query("SELECT strftime('%m',invoice_date) as month, count(*) as trans "
                       "FROM data "
                  "group by strftime('%m',invoice_date)"
                       , disk_engine)

pd.read_sql_query("SELECT invoice_date, count(*) as trans "
                       "FROM data "
                  "group by  invoice_date"
                       , disk_engine)

pd.read_sql_query("SELECT strftime('%W',invoice_date) as week, count(*) as trans "
                       "FROM data "
                  "group by strftime('%W',invoice_date)"
                       , disk_engine)

,week,trans
0,01,7670
1,02,8840
2,03,9395
3,04,7733
4,05,6994
5,06,5734
6,07,7071
7,08,7680
8,09,7373
9,10,7475


In [226]:
pd.read_sql_query("SELECT CustomerID, strftime('%m',invoice_date) as month, "
                  " strftime('%Y',invoice_date) as year, count(distinct InvoiceNo) as orders "
                  "from data " 
                  " where CustomerID in (select CustomerID from data where invoice_date <=  '2010-12-31' ) "
                  " and invoice_date >=  '2011-01-01' "
                  " group by CustomerID, strftime('%m',invoice_date), strftime('%Y',invoice_date)"
                  
                       , disk_engine)

,CustomerID,month,year,orders
0,12347.0,01,2011,1
1,12347.0,04,2011,1
2,12347.0,06,2011,1
3,12347.0,08,2011,1
4,12347.0,10,2011,1
5,12347.0,12,2011,1
6,12348.0,01,2011,1
7,12348.0,04,2011,1
8,12348.0,09,2011,1
9,12370.0,03,2011,1


In [237]:
aug11_custs.tail()

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoice_date
362580,541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/11 12:50,0.85,12680.0,France,2011-12-09
362581,541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/11 12:50,2.10,12680.0,France,2011-12-09
362582,541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/11 12:50,4.15,12680.0,France,2011-12-09
362583,541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/11 12:50,4.15,12680.0,France,2011-12-09
362584,541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/11 12:50,4.95,12680.0,France,2011-12-09


In [27]:
#concat descriptions for a customer in a distinct month
def concat_text (df):
    #text_concat=[row.Description.split() for idx, row in df.iterrows()]
    return " ".join(df.Description)
    return text_concat
    
    
    return text_concat

def make_seq_text (df):
    text_concat=[row.description_concat for idx, row in df.iterrows()]
    return text_concat



description_text=aug11_custs.groupby(['CustomerID','month']).apply(concat_text)
pd.set_option('max_colwidth', 8000)
agg_desc=pd.DataFrame(description_text.reset_index())
agg_desc.columns=['CustomerID','month','description_concat']
agg_desc.head()

#description_text_single=agg_desc.groupby(['CustomerID']).apply(make_seq_text)
#description_text_single=pd.DataFrame(description_text_single.reset_index())
#description_text_single.head()

#target
#agg_desc.join(Sep11_custs,on=)


,CustomerID,month,description_concat
0,12346.0,01,MEDIUM CERAMIC TOP STORAGE JAR MEDIUM CERAMIC TOP STORAGE JAR
1,12347.0,01,PINK NEW BAROQUECANDLESTICK CANDLE BLUE NEW BAROQUE CANDLESTICK CANDLE BLACK CANDELABRA T-LIGHT HOLDER WOODLAND CHARLOTTE BAG AIRLINE BAG VINTAGE JET SET BROWN AIRLINE BAG VINTAGE JET SET WHITE SANDWICH BATH SPONGE ALARM CLOCK BAKELIKE CHOCOLATE ALARM CLOCK BAKELIKE GREEN ALARM CLOCK BAKELIKE RED ALARM CLOCK BAKELIKE PINK ALARM CLOCK BAKELIKE ORANGE SMALL HEART MEASURING SPOONS 72 SWEETHEART FAIRY CAKE CASES 60 TEATIME FAIRY CAKE CASES PACK OF 60 MUSHROOM CAKE CASES PACK OF 60 SPACEBOY CAKE CASES TEA TIME OVEN GLOVE RED RETROSPOT OVEN GLOVE RED RETROSPOT OVEN GLOVE DOUBLE SET/2 RED RETROSPOT TEA TOWELS REGENCY CAKESTAND 3 TIER BOX OF 6 ASSORTED COLOUR TEASPOONS MINI LADLE LOVE HEART RED CHOCOLATE CALCULATOR TOOTHPASTE TUBE PEN SET OF 2 TINS VINTAGE BATHROOM RED TOADSTOOL LED NIGHT LIGHT 3D DOG PICTURE PLAYING CARDS
2,12347.0,04,AIRLINE BAG VINTAGE JET SET WHITE AIRLINE BAG VINTAGE JET SET RED AIRLINE BAG VINTAGE TOKYO 78 AIRLINE BAG VINTAGE JET SET BROWN RED RETROSPOT PURSE ICE CREAM SUNDAE LIP GLOSS VINTAGE HEADS AND TAILS CARD GAME HOLIDAY FUN LUDO TREASURE ISLAND BOOK BOX WATERING CAN PINK BUNNY RED DRAWER KNOB ACRYLIC EDWARDIAN LARGE HEART MEASURING SPOONS SMALL HEART MEASURING SPOONS PACK OF 60 DINOSAUR CAKE CASES RED RETROSPOT OVEN GLOVE DOUBLE REGENCY CAKESTAND 3 TIER ROSES REGENCY TEACUP AND SAUCER RED TOADSTOOL LED NIGHT LIGHT MINI PAINT SET VINTAGE 3D SHEET OF DOG STICKERS 3D SHEET OF CAT STICKERS SMALL FOLDING SCISSOR(POINTED EDGE) GIFT BAG PSYCHEDELIC APPLES SET OF 2 TINS VINTAGE BATHROOM
3,12347.0,06,"RABBIT NIGHT LIGHT REGENCY TEA STRAINER REGENCY TEA PLATE GREEN REGENCY TEA PLATE PINK REGENCY TEA PLATE ROSES REGENCY TEAPOT ROSES REGENCY SUGAR BOWL GREEN REGENCY MILK JUG PINK AIRLINE BAG VINTAGE TOKYO 78 AIRLINE BAG VINTAGE JET SET BROWN VICTORIAN SEWING KIT NAMASTE SWAGAT INCENSE TRIPLE HOOK ANTIQUE IVORY ROSE SMALL HEART MEASURING SPOONS 3D DOG PICTURE PLAYING CARDS FEATHER PEN,COAL BLACK ALARM CLOCK BAKELIKE RED ALARM CLOCK BAKELIKE CHOCOLATE"
4,12347.0,08,SET OF 60 VINTAGE LEAF CAKE CASES SET 40 HEART SHAPE PETIT FOUR CASES AIRLINE BAG VINTAGE JET SET BROWN AIRLINE BAG VINTAGE JET SET RED AIRLINE BAG VINTAGE JET SET WHITE AIRLINE BAG VINTAGE TOKYO 78 AIRLINE BAG VINTAGE WORLD CHAMPION WOODLAND DESIGN COTTON TOTE BAG WOODLAND CHARLOTTE BAG ALARM CLOCK BAKELIKE RED TRIPLE HOOK ANTIQUE IVORY ROSE SINGLE ANTIQUE ROSE HOOK IVORY TEA TIME OVEN GLOVE 72 SWEETHEART FAIRY CAKE CASES 60 TEATIME FAIRY CAKE CASES PACK OF 60 DINOSAUR CAKE CASES REGENCY CAKESTAND 3 TIER REGENCY MILK JUG PINK 3D DOG PICTURE PLAYING CARDS REVOLVER WOODEN RULER VINTAGE HEADS AND TAILS CARD GAME RED REFECTORY CLOCK


In [ ]:
"""
Split training and validation

create 



"""

In [307]:
12358.0 	[[FAIRY CAKE DESIGN UMBRELLA CERAMIC STRAWBERRY DESIGN MUG CERAMIC CAKE STAND + HANGING CAKES CERAMIC CAKE DESIGN SPOTTED PLATE DOORMAT FAIRY CAKE EDWARDIAN PARASOL PINK EDWARDIAN PARASOL NATURAL EDWARDIAN PARASOL RED EDWARDIAN PARASOL BLACK STRAWBERRY CERAMIC TRINKET BOX CERAMIC BOWL WITH STRAWBERRY DESIGN POSTAGE], [EDWARDIAN PARASOL RED EDWARDIAN PARASOL NATURAL EDWARDIAN PARASOL BLACK EDWARDIAN PARASOL PINK CERAMIC STRAWBERRY DESIGN MUG CERAMIC STRAWBERRY CAKE MONEY BANK POSTAGE]]

[['a', 'b'], ['c', 'd']]

In [322]:
full.loc[full.CustomerID ==12346.0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoice_date
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,1/18/11 10:01,1.04,12346.0,United Kingdom,2011-01-18
61624,541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,1/18/11 10:17,1.04,12346.0,United Kingdom,2011-01-18


In [5]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                              'foo', 'bar', 'foo', 'foo'],
                       'B' : ['one', 'one', 'two', 'three',
                              'two', 'two', 'one', 'three'],
                       'C' : np.random.randn(8),
                       'D' : np.random.randn(8)})

In [16]:
def showme(df):
    print df
    print df.C.mean()
    print 'F'
    return df.C.mean()


df.groupby(['A']).apply(showme)

     A      B         C         D
1  bar    one -0.176623  0.401817
3  bar  three  2.417999 -1.041640
5  bar    two -0.063081 -0.199995
0.726098576597
F
     A      B         C         D
1  bar    one -0.176623  0.401817
3  bar  three  2.417999 -1.041640
5  bar    two -0.063081 -0.199995
0.726098576597
F
     A      B         C         D
0  foo    one -0.140129  1.008305
2  foo    two -0.670310  0.638910
4  foo    two -0.833797  1.178657
6  foo    one  0.310851  0.535189
7  foo  three  1.228427  0.243668
-0.0209915851557
F


A
bar    0.726099
foo   -0.020992
dtype: float64